#### Imports & Constants

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pathlib import Path

# Constants
BASE_URL = "https://con.2merkato.com/prices"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
}

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

#### Fetch Main Page & List Categories

In [2]:
def fetch_page(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

soup = fetch_page(BASE_URL)
if soup:
    print("Page title:", soup.title.string if soup.title else "No title")
    
    # Find categories (they are in h5 or links under sections)
    categories = []
    for section in soup.find_all(['h3', 'h4', 'h5']):  # adjust selector based on inspection
        cat_link = section.find('a')
        if cat_link:
            cat_name = cat_link.get_text(strip=True)
            cat_url = cat_link['href']
            if '/prices/cat/' in cat_url:
                categories.append({'name': cat_name, 'url': 'https://con.2merkato.com' + cat_url})
    
    print(f"Found {len(categories)} categories")
    pd.DataFrame(categories).head(10)  # display

Page title: Construction Market Watch
Found 12 categories


#### Fetch & Extract Sample Items from Concrete Work

In [4]:
import re

CONCRETE_URL = "https://con.2merkato.com/prices/cat/2"

soup_cat = fetch_page(CONCRETE_URL)
if soup_cat:
    table = soup_cat.find('table')  # main table
    if table:
        rows = table.find_all('tr')[1:]  # skip header
        items = []
        
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 5:
                # Material name & link
                name_cell = cells[0]
                name_link = name_cell.find('a')
                if name_link:
                    material = name_link.get_text(strip=True)
                    href = name_link['href']
                    detail_url = 'https://con.2merkato.com' + href if href.startswith('/') else href
                    
                    # Price & unit
                    price_text = cells[1].get_text(strip=True)
                    
                    # Last checked date
                    last_checked = cells[4].get_text(strip=True)
                    
                    # Extract numeric price and unit with regex
                    price_match = re.search(r'([\d,]+\.\d{2})\s*Br\s*per\s*(\w+)', price_text)
                    price_etb = price_match.group(1).replace(',', '') if price_match else None
                    unit = price_match.group(2) if price_match else None
                    
                    items.append({
                        'material': material,
                        'price_text': price_text,
                        'price_etb': float(price_etb) if price_etb else None,
                        'unit': unit,
                        'last_checked': last_checked,
                        'detail_url': detail_url,
                        'scraped_at': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
                    })
        
        df_concrete = pd.DataFrame(items)
        print(f"Extracted {len(df_concrete)} items cleanly")
        display(df_concrete.head(10))  # show first 10
        
        # Save enhanced version
        df_concrete.to_csv(DATA_DIR / "concrete_work_prices_clean_2026.csv", index=False)
        print("Saved enhanced data to data/concrete_work_prices_clean_2026.csv")
    else:
        print("No table found — check page structure or selectors.")
time.sleep(2)

Extracted 35 items cleanly


,material,price_text,price_etb,unit,last_checked,detail_url,scraped_at
0,Cement - OPC Mugar,720.00 Brper quintal,720.0,quintal,"Oct 16, 2021",https://con.2merkato.com/prices/material/2/3,2026-02-18 10:39:44
1,Cement - PPC Mugar,"1,500.00 Brper quintal",1500.0,quintal,"Aug 10, 2024",https://con.2merkato.com/prices/material/2/14,2026-02-18 10:39:44
2,Cement - PPC Mesebo,"2,000.00 Brper kg",2000.0,kg,"Mar 25, 2023",https://con.2merkato.com/prices/material/2/15,2026-02-18 10:39:44
3,Cement - OPC Mesebo,320.00 Brper quintal,320.0,quintal,"Nov 28, 2018",https://con.2merkato.com/prices/material/2/16,2026-02-18 10:39:44
4,Cement - PPC Derba,"1,500.00 Brper quintal",1500.0,quintal,"Aug 10, 2024",https://con.2merkato.com/prices/material/2/17,2026-02-18 10:39:44
5,Cement - OPC Derba,630.00 Brper quintal,630.0,quintal,"Dec 25, 2021",https://con.2merkato.com/prices/material/2/18,2026-02-18 10:39:44
6,Cement - PPC Dangote,"1,580.00 Brper quintal",1580.0,quintal,"Aug 10, 2024",https://con.2merkato.com/prices/material/2/19,2026-02-18 10:39:44
7,Cement - OPC Dangote,700.00 Brper quintal,700.0,quintal,"Nov 20, 2021",https://con.2merkato.com/prices/material/2/20,2026-02-18 10:39:44
8,Sand (hauling distance less than 5km in Addis ...,"3,000.00 Brper m3",3000.0,m3,"Aug 10, 2024",https://con.2merkato.com/prices/material/2/21,2026-02-18 10:39:44
9,Sand (hauling distance more than 5km in Addis ...,"3,200.00 Brper m3",3200.0,m3,"Aug 10, 2024",https://con.2merkato.com/prices/material/2/22,2026-02-18 10:39:44


Saved enhanced data to data/concrete_work_prices_clean_2026.csv
